In [2]:
import pandas as pd
import numpy as np
#import sframe
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model
import functools
import json

In [13]:
product_pd = pd.read_csv("/Users/wxu/workspace/ML-UW-Coursera/classification/week1/amazon_baby.csv",header=0)

In [4]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

In [14]:
##data cleaning
product_pd = product_pd.fillna({"review":''})
product_pd["review_clean"] = product_pd["review"].apply(remove_punctuation)
product_pd = product_pd[product_pd['rating'] != 3]
product_pd['sentiment'] = product_pd['rating'].apply(lambda rating: +1 if rating>3 else -1)

In [16]:
#split to train/test

with open("/Users/wxu/workspace/ML-UW-Coursera/classification/week1/trainingIdx.json") as myjson:
    for line in myjson:
        trainIdx = json.loads(line)
with open("/Users/wxu/workspace/ML-UW-Coursera/classification/week1/testingIdx.json") as myjson:
    for line in myjson:
        testIdx = json.loads(line)
train = product_pd.iloc[trainIdx]
test = product_pd.iloc[testIdx]
train = train.reset_index(range(len(train))).drop("index",axis=1)
test = test.reset_index(range(len(train))).drop("index",axis=1)
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train['review_clean'])
test_matrix = vectorizer.transform(test['review_clean'])


In [72]:
logitRgrssn = sklearn.linear_model.LogisticRegression()
sentimentModel = logitRgrssn.fit(train_matrix,train['sentiment'])

In [73]:
#quiz
coef = sentimentModel.coef_.flatten()
pos = (coef>=0).sum()
pos

86121

In [76]:
sample_test_data = test[10:13]
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])




In [77]:
scores = pd.Series(sentimentModel.decision_function(sample_test_matrix))
print scores

0     5.609118
1    -3.146160
2   -10.417916
dtype: float64


In [78]:
def decision_label(scores):
    labels =scores.apply(lambda score:+1 if score>=0 else -1)
    return labels

In [80]:
labels = decision_label(scores)
print labels

0    1
1   -1
2   -1
dtype: int64


In [81]:
probs = 1/(1+np.exp(-scores))
print probs

0    0.996349
1    0.041243
2    0.000030
dtype: float64


In [88]:
test["predScore"] = sentimentModel.decision_function(test_matrix)
test["predLabel"] = sentimentModel.predict(test_matrix)
test["predProbs"] = sentimentModel.predict_proba(test_matrix)[:,1]

In [92]:
(test["predLabel"]==test["sentiment"]).sum()

31077

In [53]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']
vectorizer_word_subset = CountVectorizer(vocabulary=significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train['review_clean'])
test_matrix_word_subset = vectorizer_word_subset.transform(test['review_clean'])

In [54]:
simple_model = logitRgrssn.fit(train_matrix_word_subset,train['sentiment'])

In [55]:
wordTable = {'word':significant_words,'coefficient':simple_model.coef_.flatten()}

In [57]:
wordTable2 = pd.DataFrame(wordTable) 

In [64]:
(test["sentiment"]==1).sum()*1.0/len(test)*1.0

0.84278257739380846

In [ ]:
# Note from the exercise
#1. when add a new column to pandas data frame, df["newcolumn"] = ndarray(len(df),). don't add a pd series to a df 
# to avoid the index messed up

#2.create dict to convert to pddf {'word':significant_words,'coefficient':simple_model.coef_.flatten()}.key is column name and value is value list
#3.count number of records that met with some condition use sum function (test["sentiment"]==1).sum()
#4.